# Webscrape notebook for DDO news bot
Alekzander Green

This notebook provides a quick investigation into the nuances of webscraping the DDO news page in order to provide a Discord call with updates regarding new content, events, and coupon codes as they are posted to the DDO news page. Once the investigation within this notebook is complete, a bot can be used to provide updates to the server.

### Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

### Setup and Constants

In [2]:
URL = "https://www.ddo.com/home/news"

options = Options()
options.add_argument("--headless=new")

### Open a webdriver

In [3]:
driver = webdriver.Chrome(options=options)

### Follow the URL

In [4]:
driver.get(URL)

### Find news cards

In [5]:
card_elements = driver.find_elements("class name", "news-link")

### Filter and collect card info

Generate dictionaries

In [6]:
cards: list[dict[str, str]] = []

for card_element in card_elements:
    card = {
        "link":card_element.get_attribute("href"),
        "title":card_element.find_element("class name", "news-title").text,
        "full_summary":card_element.find_element("class name", "news-summary").get_attribute("title"),
        "short_summary":card_element.find_element("class name", "news-summary").text,
        "date":card_element.find_element("class name", "news-date").text.split(" | ")[0],
        "coupon":None
    }

    cards.append(card)

Preview

In [7]:
cards

[{'link': 'https://www.ddo.com/news/ddo-myth-drannor-release-date',
  'title': 'Magic of Myth Drannor Release Date Information',
  'full_summary': "We are getting ready to release Magic of Myth Drannor and we want to share an updated release date with you today. Magic of Myth Drannor will release on Wednesday, August 14th, 2024. We currently have a loot preview happening on Lamannia, and we're planning for another preview later this month. We look forward to sharing more of our next expansion with you in the coming weeks!",
  'short_summary': 'We are getting ready to release Magic of Myth Drannor and we want to share an updated release date…',
  'date': 'Thu Jul 11 2024',
  'coupon': None},
 {'link': 'https://www.ddo.com/news/ddo-sales-071124',
  'title': 'Sales: July 11th - 18th!',
  'full_summary': 'Get the latest sales here!',
  'short_summary': 'Get the latest sales here!',
  'date': 'Thu Jul 11 2024',
  'coupon': None},
 {'link': 'https://www.ddo.com/news/ddo-bonus-071124',
  'tit

Get coupon codes

In [8]:
for card in cards:
    if "sales:" in card["title"].lower():
        driver.get(card["link"])
        
        body = driver.find_element("class name", "article-body")
        sales_text_lines = body.text.splitlines()

        coupon = {}

        line: str
        for i, line in enumerate(sales_text_lines):
            l_line = line.lower().strip()
            if "weekly" in l_line and "coupon" in l_line:
                coupon["item"] = sales_text_lines[i+1].strip()
                coupon["code"] = sales_text_lines[i+2].strip().split(": ")[1]
                coupon["exp"] = sales_text_lines[i+3].rstrip("!").lstrip("Now through ")

        card["coupon"] = coupon

In [9]:
cards

[{'link': 'https://www.ddo.com/news/ddo-myth-drannor-release-date',
  'title': 'Magic of Myth Drannor Release Date Information',
  'full_summary': "We are getting ready to release Magic of Myth Drannor and we want to share an updated release date with you today. Magic of Myth Drannor will release on Wednesday, August 14th, 2024. We currently have a loot preview happening on Lamannia, and we're planning for another preview later this month. We look forward to sharing more of our next expansion with you in the coming weeks!",
  'short_summary': 'We are getting ready to release Magic of Myth Drannor and we want to share an updated release date…',
  'date': 'Thu Jul 11 2024',
  'coupon': None},
 {'link': 'https://www.ddo.com/news/ddo-sales-071124',
  'title': 'Sales: July 11th - 18th!',
  'full_summary': 'Get the latest sales here!',
  'short_summary': 'Get the latest sales here!',
  'date': 'Thu Jul 11 2024',
  'coupon': {'item': 'Free Potion of Rage x5',
   'code': 'WHATWOULDGROGDO',
   